In [45]:
using SparseArrays, Polynomials, PyCall, LinearAlgebra
using Plots
default(legend=false, linewidth=2)

In [3]:
"""
    mkanim(x, allu; [filename, axis])

Utility for animating PDE solutions.
"""
function mkanim(x, allu; filename=tempname() * ".mp4", axis=[0,1,-0.1,1.1])
    anim = @animate for i in 1:length(allu)
        plot(xlims=axis[1:2], ylims=axis[3:4])
        plot!(x[1], allu[i][1], linecolor=:blue)
        plot!(x[2], allu[i][2], linecolor=:black)
    end
    gif(anim, filename)
end

mkanim

In [82]:
"""
    function gauss_quad(p)

Gaussian quadrature on [-1,1] for given degree of precision `p`
"""
function gauss_quad(p)
    n = ceil((p+1)/2)
    b = 1:n-1
    b = @. b / sqrt(4*b^2 - 1)
    eval, evec = eigen(diagm(1 => b, -1 => b))
    return eval, 2*evec[1,:].^2
end

"""
    function legendre_poly(x, p)

Legendre polynomials and derivatives up to degree `p` at nodes `x`
"""
function legendre_poly(x, p)
    z = zeros(size(x))
    o = ones(size(x))
    y = hcat(o, x, repeat(z, 1, p-1))
    dy = hcat(z, o, repeat(z, 1, p-1))
    for i = 1:p-1
        @. y[:,i+2] = ((2i+1)*x*y[:,i+1] - i*y[:,i]) / (i+1)
        @. dy[:,i+2] = ((2i+1)*(x*dy[:,i+1] + y[:,i+1]) - i*dy[:,i]) / (i+1)
    end
    y, dy
end

xx = -1:0.01:1
p = 3
h = 0.1
s0 = [-cos(pi*i/p) for i=0:p] 
s = @. (s0 + 1) * h/2
gx, gw = gauss_quad(2*p)

yy,dyy = legendre_poly(gx, p)
#plot(gx, yy)

([1.0 -0.86113631159405 0.6123336207187071 -0.30474698495519564; 1.0 -0.3399810435848556 -0.32661933500442875 0.4117279996728991; 1.0 0.33998104358485715 -0.3266193350044272 -0.41172799967290014; 1.0 0.8611363115940526 0.6123336207187138 0.3047469849552061], [0.0 1.0 -2.58340893478215 4.061668103593536; 0.0 1.0 -1.0199431307545668 -0.6330966750221438; 0.0 1.0 1.0199431307545714 -0.633096675022136; 0.0 1.0 2.583408934782158 4.061668103593569])

In [85]:
function dgconvect(; n=10, p=1, T=1.0, dt=1e-3)
    # Discretization
    h = 1 / n
    #s = @. (0:p) / p
    s0 = [-cos(pi*i/p) for i=0:p] 
    s = @. (s0 + 1) * h/2 
    x = @. s + (0:h:1-h)'

    # Gaussian initial condition (and exact solution if shifted)
    uinit(x) = exp(-(x - 0.5)^2 / 0.1^2)


    yy, dyy = legendre_poly(s0, p)
    C = inv(yy)
    
    # Evaluate Legendre polynomials at Gaussian nodes
    gx, gw = gauss_quad(2*p)
    gyy, gdyy = legendre_poly(gx, p)

    # Basis function number i evaluated at Gaussian nodes
    phi(i) = [dot(C[:,i], gyy[j,:]) for j=1:p+1]
    # Scale derivative 
    phider(i) = [dot(C[:,i], gdyy[j,:]) for j=1:p+1] .* 2 / h 

    Mel = h/2 .* [dot(phi(i) .* phi(j), gw) for i=1:p+1, j=1:p+1]
    Kel = h/2 .* [dot(phider(i) .* phi(j), gw) for i=1:p+1, j=1:p+1]
     # RHS function in semi-discrete system
    function rhs(u)
        r = Kel * u
        r[end,:] = r[end,:] - u[end,:]
        r[1,:] = r[1,:] + u[end, [end; 1:end-1]]
        r = Mel \ r
    end

    # Setup
    u = uinit.(x)
    nsteps = round(Int, T/dt)

    # Setup plotting
    xx = (0:0.01:1) # For exact solution
    allu = [ (u, uinit.(xx)) ]

    # Main loop
    for it = 1:nsteps
        # Runge-Kutta 4
        k1 = dt * rhs(u)
        k2 = dt * rhs(u + k1/2)
        k3 = dt * rhs(u + k2/2)
        k4 = dt * rhs(u + k3)
        u += (k1 + 2*k2 + 2*k3 + k4) / 6

        # Plotting
        if mod(it, round(nsteps/100)) == 0
            uexact = @. uinit(mod(xx - dt*it, 1.0))
            push!(allu, (u, uexact))
        end
    end
    uexact = @. uinit(mod(x - T, 1.0))  # Exact final solution
    error = maximum(abs.(u - uexact))   # Discrete inf-norm error""
    display(u)
    display(uexact)
    
    return (x,xx), allu, error
end

dgconvect (generic function with 1 method)

In [86]:
x, allu, error = dgconvect(p=3, n=10)
#mkanim(x, allu)

4×10 Matrix{Float64}:
 -8.36599e-6  -2.93541e-5   -0.000786397  …   3.83688e-5  -4.20342e-5
 -8.80651e-7  -3.49237e-5    0.00124777       4.82316e-5  -3.95331e-6
 -5.37151e-6   8.19814e-5    0.00578746      -1.52027e-5   6.72809e-6
 -2.12937e-5   0.000308241   0.0171487       -3.51397e-5  -4.18507e-6

4×10 Matrix{Float64}:
 1.38879e-11  1.12535e-7  0.00012341   0.0183156  …  0.00012341  1.12535e-7
 1.58939e-10  7.81149e-7  0.000519575  0.0467706     2.58681e-5  1.43072e-8
 1.43072e-8   2.58681e-5  0.00632972   0.209611      7.81149e-7  1.58939e-10
 1.12535e-7   0.00012341  0.0183156    0.367879      1.12535e-7  1.38879e-11

(([0.0 0.1 … 0.8 0.9; 0.024999999999999994 0.125 … 0.8250000000000001 0.925; 0.075 0.175 … 0.875 0.975; 0.1 0.2 … 0.9 1.0], 0.0:0.01:1.0), [([1.388794386496407e-11 1.1253517471925912e-7 … 0.00012340980408667956 1.1253517471925912e-7; 1.5893910094516422e-10 7.811489408304519e-7 … 2.5868100222654076e-5 1.4307241918567688e-8; 1.430724191856779e-8 2.5868100222654168e-5 … 7.811489408304519e-7 1.5893910094516422e-10; 1.1253517471925912e-7 0.00012340980408667978 … 1.1253517471925912e-7 1.388794386496407e-11], [1.388794386496407e-11, 3.7375713279442825e-11, 9.859505575991552e-11, 2.5493818803919876e-10, 6.461431773106131e-10, 1.6052280551856172e-9, 3.908938434264878e-9, 9.330287574505038e-9, 2.1829577951254933e-8, 5.006218020767049e-8  …  5.006218020767049e-8, 2.182957795125478e-8, 9.330287574505005e-9, 3.908938434264892e-9, 1.605228055185623e-9, 6.461431773106154e-10, 2.5493818803919876e-10, 9.859505575991552e-11, 3.7375713279442825e-11, 1.388794386496407e-11]), ([4.122408848823318e-7 -4.4462